In [ ]:
import  numpy as np
import pandas as pd

In [ ]:
import string
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer

In [ ]:
from nltk.util import ngrams
from collections import Counter
from gensim.models import word2vec

In [ ]:
data = pd.read_csv('../input/ziphack/train_file.csv')
test = pd.read_csv('../input/ziphack/test_file.csv')

In [ ]:
data["MaterialType"] = data['MaterialType'].str.replace('[^\w\s]','')

In [ ]:
data.drop(['UsageClass','CheckoutType','CheckoutYear','CheckoutMonth','ID'],inplace=True,axis=1)
test.drop(['UsageClass','CheckoutType','CheckoutYear','CheckoutMonth'],inplace=True,axis=1)

In [ ]:
y = data["MaterialType"]

In [ ]:
data = data.apply(lambda x: x.astype(str).str.lower())
test = test.apply(lambda x: x.astype(str).str.lower())

In [ ]:
data['MaterialType'] = data['MaterialType'].str.upper()
data.head()

In [ ]:
data["Title"] = data['Title'].str.replace('[^\w\s]','')
test["Title"] = test['Title'].str.replace('[^\w\s]','')
data["Creator"] = data['Creator'].str.replace('[^\w\s]','')
data["Subjects"] = data['Subjects'].str.replace('[^\w\s]','')
##data["Subjects"] = data['Subjects'].str.replace('[0-9]','')
data["Publisher"] = data['Publisher'].str.replace('[^\w\s]','')
data["PublicationYear"] = data['PublicationYear'].str.replace('[^\w\s]','')
data["PublicationYear"] = data['PublicationYear'].str.replace('[cp]','')
data["Creator"] = data['Creator'].str.replace('[0-9]','')
test["Subjects"] = test['Subjects'].str.replace('[^\w\s]','')
##test["Subjects"] = test['Subjects'].str.replace('[0-9]','')

In [ ]:
from nltk.corpus import stopwords

In [ ]:
sw = stopwords.words('english')

In [ ]:
def stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    return " ".join(text)

In [ ]:
data['Title'] = data['Title'].apply(stopwords)
data['Subjects'] = data['Subjects'].apply(stopwords)
test['Subjects'] = test['Subjects'].apply(stopwords)

In [ ]:
print(data.nunique())

In [ ]:
import nltk

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
stemmer = PorterStemmer()

In [ ]:
def pre(data):
    word = lemmatizer.lemmatize(data) 
    return word

In [ ]:
data['Title'] = data['Title'].apply(pre)
data['Subjects'] = data['Subjects'].apply(pre)
test['Subjects'] = test['Subjects'].apply(pre)
test['Title'] = test['Title'].apply(pre)

In [ ]:
print(data.nunique())

In [ ]:
data["Subjects"] = data['Subjects'].replace('nan','')

In [ ]:
data["Phrase"] = data["Title"].map(str) + ' ' + data["Subjects"] 
test["Phrase"] = test["Title"].map(str) + ' ' + test["Subjects"]

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
led = LabelEncoder()
led.fit(data["MaterialType"])
data["MaterialType"] = led.transform(data["MaterialType"].values)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from scipy.sparse import hstack, csr_matrix
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
x = data['Phrase']
w = test['Phrase']

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(x)

In [ ]:
X = vectorizer.transform(x)
print('Shape of Sparse Matrix: ', X.shape)
print('Amount of Non-Zero occurrences: ', X.nnz)
# Percentage of non-zero values
density = (100.0 * X.nnz / (X.shape[0] * X.shape[1]))
print('Density: {}'.format((density)))

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(x)

In [ ]:
W = vectorizer.transform(w)
print('Shape of Sparse Matrix: ', W.shape)
print('Amount of Non-Zero occurrences: ', W.nnz)
# Percentage of non-zero values
density = (100.0 * W.nnz / (W.shape[0] * W.shape[1]))
print('Density: {}'.format((density)))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=100)
rfc = RandomForestClassifier(min_samples_split = 10, n_estimators = 200, verbose = 1, n_jobs = -1,warm_start = True)
rfc.fit(X_train, y_train)

# Testing the model
predictions = rfc.predict(X_test)
print(classification_report(y_test, predictions))

In [ ]:
y_pred = rfc.predict(X_test)

In [ ]:
pred = rfc.predict(W)

In [ ]:
submission0 = pd.DataFrame({'ID':test['ID'],'MaterialType': pred})
#submission0["MaterialType"] = led.inverse_transform(submission0["MaterialType"])
#submission0 = submission0.apply(lambda x: x.astype(str).str.upper())
submission0.to_csv('submission_rf.csv', index=False)

led.fit(pred)
pred = led.transform(pred)

In [ ]:
lr = LogisticRegression(C=1.2,class_weight = 'balanced')
lr.fit(X_train, y_train)

# Testing the model
predictions = rfc.predict(X_test)
print(classification_report(y_test, predictions))

In [ ]:
preds = rfc.predict(W)

In [ ]:
submissions = pd.DataFrame({'ID':test['ID'],'MaterialType': preds})
#submissions["MaterialType"] = led.inverse_transform(submissions["MaterialType"])
#submissions = submissions.apply(lambda x: x.astype(str).str.upper())
submissions.to_csv('submission_lr.csv', index=False)
#submissions.head()
led.fit(preds)
preds = led.transform(preds)

In [ ]:
pred

In [ ]:
import numpy as np 
import pandas as pd 
import nltk
import os
import gc
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
#pd.set_option('display.max_colwidth',100)
pd.set_option('display.max_colwidth', -1)

In [ ]:
gc.collect()

In [ ]:
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import SnowballStemmer,WordNetLemmatizer
stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()
from string import punctuation
import re

In [ ]:
def clean_review(review_col):
    review_corpus=[]
    for i in range(0,len(review_col)):
        review=str(review_col[i])
        review=re.sub('[^a-zA-Z]',' ',review)
        #review=[stemmer.stem(w) for w in word_tokenize(str(review).lower())]
        review=[lemma.lemmatize(w) for w in word_tokenize(str(review).lower())]
        review=' '.join(review)
        review_corpus.append(review)
    return review_corpus

In [ ]:
data['clean_review']=clean_review(data.Phrase.values)
test['clean_review']=clean_review(test.Phrase.values)

In [ ]:
train_text=data.clean_review.values
test_text=test.clean_review.values
target=data.MaterialType.values
y=to_categorical(target)
print(train_text.shape,target.shape,y.shape)

In [ ]:
X_train_text,X_val_text,y_train,y_val=train_test_split(train_text,y,test_size=0.2,stratify=y,random_state=123)
print(X_train_text.shape,y_train.shape)
print(X_val_text.shape,y_val.shape)

In [ ]:
all_words=' '.join(X_train_text)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
num_unique_word=len(dist)
num_unique_word

In [ ]:
r_len=[]
for text in X_train_text:
    word=word_tokenize(text)
    l=len(word)
    r_len.append(l)
    
MAX_REVIEW_LEN=np.max(r_len)
MAX_REVIEW_LEN

In [ ]:
max_features = num_unique_word
max_words = MAX_REVIEW_LEN
batch_size = 128
epochs = 5
num_classes=8

In [ ]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train_text))
X_train = tokenizer.texts_to_sequences(X_train_text)
X_val = tokenizer.texts_to_sequences(X_val_text)
X_test = tokenizer.texts_to_sequences(test_text)

In [ ]:
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_val = sequence.pad_sequences(X_val, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)
print(X_train.shape,X_val.shape,X_test.shape)

In [ ]:
model1=Sequential()
model1.add(Embedding(max_features,100,mask_zero=True))
model1.add(LSTM(64,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model1.add(LSTM(32,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model1.add(Dense(num_classes,activation='softmax'))
model1.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model1.summary()

In [ ]:
%%time
history1=model1.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

In [ ]:
y_pred1=model1.predict_classes(X_test,verbose=1)

In [ ]:
submission = pd.DataFrame({'ID':test['ID'],'MaterialType': y_pred1})
submission["MaterialType"] = led.inverse_transform(submission["MaterialType"])
submission = submission.apply(lambda x: x.astype(str).str.upper())
submission.to_csv('submission_lstm.csv', index=False)

In [ ]:
model2= Sequential()
model2.add(Embedding(max_features,100,input_length=max_words))
model2.add(Dropout(0.2))

model2.add(Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
model2.add(GlobalMaxPooling1D())

model2.add(Dense(128,activation='relu'))
model2.add(Dropout(0.2))

model2.add(Dense(num_classes,activation='softmax'))


model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model2.summary()

In [ ]:
%%time
history2=model2.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

In [ ]:
y_pred2=model2.predict_classes(X_test, verbose=1)

In [ ]:
submission1 = pd.DataFrame({'ID':test['ID'],'MaterialType': y_pred2})
submission1["MaterialType"] = led.inverse_transform(submission1["MaterialType"])
submission1 = submission1.apply(lambda x: x.astype(str).str.upper())
submission1.to_csv('submission_cnn_new.csv', index=False)

In [ ]:
model3= Sequential()
model3.add(Embedding(max_features,100,input_length=max_words))
model3.add(Conv1D(64,kernel_size=3,padding='same',activation='relu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Dropout(0.25))
model3.add(GRU(128,return_sequences=True))
model3.add(Dropout(0.3))
model3.add(Flatten())
model3.add(Dense(128,activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(8,activation='softmax'))
model3.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model3.summary()

In [ ]:
%%time
history3=model3.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

In [ ]:
y_pred3=model3.predict_classes(X_test, verbose=1)

In [ ]:
submission2 = pd.DataFrame({'ID':test['ID'],'MaterialType': y_pred3})
submission2["MaterialType"] = led.inverse_transform(submission2["MaterialType"])
submission2 = submission2.apply(lambda x: x.astype(str).str.upper())
submission2.to_csv('submission_grucnn.csv', index=False)

In [ ]:
model4 = Sequential()

model4.add(Embedding(max_features, 100, input_length=max_words))
model4.add(SpatialDropout1D(0.25))
model4.add(Bidirectional(GRU(128)))
model4.add(Dropout(0.5))

model4.add(Dense(8, activation='softmax'))
model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model4.summary()

In [ ]:
%%time
history4=model4.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=epochs, batch_size=batch_size, verbose=1)

In [ ]:
y_pred4=model4.predict_classes(X_test, verbose=1)

In [ ]:
submission3 = pd.DataFrame({'ID':test['ID'],'MaterialType': y_pred4})
submission3["MaterialType"] = led.inverse_transform(submission3["MaterialType"])
submission3 = submission3.apply(lambda x: x.astype(str).str.upper())
submission3.to_csv('submission_bicnn.csv', index=False)

In [ ]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')
    
def get_embed_mat(EMBEDDING_FILE, max_features,embed_dim):
    # word vectors
    embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE, encoding='utf8'))
    print('Found %s word vectors.' % len(embeddings_index))

    # embedding matrix
    word_index = tokenizer.word_index
    num_words = min(max_features, len(word_index) + 1)
    all_embs = np.stack(embeddings_index.values()) #for random init
    embedding_matrix = np.random.normal(all_embs.mean(), all_embs.std(), 
                                        (num_words, embed_dim))
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    max_features = embedding_matrix.shape[0]
    
    return embedding_matrix

In [ ]:
EMBEDDING_FILE = '../input/glove6b100dtxt/glove.6B.100d.txt'
embed_dim = 100 #word vector dim
embedding_matrix = get_embed_mat(EMBEDDING_FILE,max_features,embed_dim)
print(embedding_matrix.shape)

In [ ]:
model5 = Sequential()
model5.add(Embedding(max_features, embed_dim, input_length=X_train.shape[1],weights=[embedding_matrix],trainable=True))
model5.add(SpatialDropout1D(0.25))
model5.add(Bidirectional(GRU(128,return_sequences=True)))
model5.add(Bidirectional(GRU(64,return_sequences=False)))
model5.add(Dropout(0.5))
model5.add(Dense(num_classes, activation='softmax'))
model5.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model5.summary()

In [ ]:
%%time
history5=model5.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=4, batch_size=batch_size, verbose=1)

In [ ]:
y_pred5=model5.predict_classes(X_test, verbose=1)

In [ ]:
submission4 = pd.DataFrame({'ID':test['ID'],'MaterialType': y_pred5})
submission4["MaterialType"] = led.inverse_transform(submission4["MaterialType"])
submission4 = submission4.apply(lambda x: x.astype(str).str.upper())
submission4.to_csv('submission_glovecnn.csv', index=False)

In [ ]:
sub_all=pd.DataFrame({'model':pred,'model0':preds,'model1':y_pred1,'model2':y_pred2,'model3':y_pred3,'model4':y_pred4,'model5':y_pred5})
pred_mode=sub_all.agg('mode',axis=1)[0].values
sub_all.head()

In [ ]:
pred_mode=[int(i) for i in pred_mode]

In [ ]:
submission5 = pd.DataFrame({'ID':test['ID'],'MaterialType': pred_mode})
submission5["MaterialType"] = led.inverse_transform(submission5["MaterialType"])
submission5 = submission5.apply(lambda x: x.astype(str).str.upper())
submission5.to_csv('submission_mean.csv', index=False)